## Multiple Charts Analysis

In [2]:
import pandas as pd
import glob2
import matplotlib.pyplot as plt
import datetime
import re
from datetime import datetime, timedelta
%matplotlib inline

In [3]:
za_the_spotify_data=pd.read_csv("Classification/all_files_ZAv1.csv")
ngr_the_spotify_data=pd.read_csv("Classification/all_files_NGv1.csv")
global_the_spotify_data=pd.read_csv("Classification/all_files_globalv1.csv")
skr_the_spotify_data=pd.read_csv("Classification/all_files_SKRv1.csv")
ind_the_spotify_data=pd.read_csv("Classification/all_files_INv1.csv")

In [4]:
za_the_spotify_data.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'artist_names',
       'track_name', 'rank', 'uri', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time', 'IsGirlGroup', 'IsBoyGroup'],
      dtype='object')

In [5]:
za_the_spotify_data.shape, ngr_the_spotify_data.shape, global_the_spotify_data.shape, skr_the_spotify_data.shape, ind_the_spotify_data.shape
# za_the_spotify_data.shape, skr_the_spotify_data.shape

((71086, 25), (13000, 25), (83782, 26), (40800, 25), (58368, 25))

#### South Africa plus Nigeria

In [7]:
zaplusng = za_the_spotify_data[["main_artist"]].merge(ngr_the_spotify_data[["main_artist"]], how='inner', on='main_artist')
zaplusng = zaplusng.drop_duplicates()

newcols = []
for acol in list(zaplusng.columns):
    newacol = acol.replace("_x", "_za")
    newacol = newacol.replace("_y", "_ng")

    newcols.append(newacol)

zaplusng.columns = newcols

zaplusng = zaplusng.drop_duplicates()

zaplusng.shape

(155, 1)

#### South Korea plus Global

In [9]:
skrplusglobal = za_the_spotify_data[["main_artist"]].merge(global_the_spotify_data[["main_artist"]], how='inner', on='main_artist')
skrplusglobal = skrplusglobal.drop_duplicates()

newcols = []
for acol in list(skrplusglobal.columns):
    newacol = acol.replace("_x", "_za")
    newacol = newacol.replace("_y", "_gbl")

    newcols.append(newacol)

skrplusglobal.columns = newcols

skrplusglobal = skrplusglobal.drop_duplicates()

skrplusglobal.shape

(613, 1)

#### South Korea plus Global plus South Africa

In [11]:
# common = list(set(zaplusglobal["lower_main_artist"].unique().tolist()) & set(skrplusglobal["lower_main_artist"].unique().tolist()))
# len(common)
# zaplusskr = za_the_spotify_data.merge(skr_the_spotify_data, how='inner', on='main_artist')
# zaplusskr = zaplusskr.drop_duplicates()

#### South Korea plus South Africa

In [13]:
zaplusskr = za_the_spotify_data[["main_artist"]].merge(skr_the_spotify_data[["main_artist"]], how='inner', on='main_artist')
zaplusskr = zaplusskr.drop_duplicates()

newcols = []
for acol in list(zaplusskr.columns):
    newacol = acol.replace("_x", "_za")
    newacol = newacol.replace("_y", "_skr")

    newcols.append(newacol)

zaplusskr.columns = newcols

zaplusskr = zaplusskr.drop_duplicates()

In [14]:
len(zaplusskr["main_artist"].unique().tolist())

284

In [15]:
zaplusskr.shape

(284, 1)

In [34]:
skrplusind = skr_the_spotify_data[["main_artist"]].merge(ind_the_spotify_data[["main_artist"]], how='inner', on='main_artist')
skrplusind = skrplusind.drop_duplicates()

newcols = []
for acol in list(skrplusind.columns):
    newacol = acol.replace("_x", "_skr")
    newacol = newacol.replace("_y", "_in")

    newcols.append(newacol)

skrplusind.columns = newcols

skrplusind = skrplusind.drop_duplicates()

### Combine ALL

In [17]:
zaplusngskrplusglobal = zaplusng.merge(skrplusglobal, how='inner', on='main_artist')
zaplusngskrplusglobal = zaplusngskrplusglobal.drop_duplicates()

In [18]:
zaplusngskrplusglobal.shape

(111, 1)

In [19]:
zaplusngskrplusglobazaplusskr = zaplusngskrplusglobal.merge(zaplusskr, how='inner', on='main_artist')
zaplusngskrplusglobazaplusskr = zaplusngskrplusglobazaplusskr.drop_duplicates()

In [36]:
zaplusngskrplusglobazaplusskrskrplusind = zaplusngskrplusglobazaplusskr.merge(skrplusind, how='inner', on='main_artist')
zaplusngskrplusglobazaplusskrskrplusind = zaplusngskrplusglobazaplusskrskrplusind.drop_duplicates()

In [38]:
combinedartists = zaplusngskrplusglobazaplusskrskrplusind["main_artist"].unique().tolist()

In [40]:
len(combinedartists)

61

In [48]:
dfnames = pd.DataFrame(combinedartists,columns=["artist"])

In [50]:
dfnames.to_csv("artistsonall5charts.csv", index=False)

In [44]:
# stop

#### Num Songs

In [ ]:
dfnumsongs = pd.DataFrame(zaplusskr.groupby("main_artist")["track_name_za"].nunique())
dfnumsongsskr = pd.DataFrame(zaplusskr.groupby("main_artist")["track_name_skr"].nunique())
dfnumsongs = dfnumsongs.reset_index()
dfnumsongsskr = dfnumsongsskr.reset_index()

In [ ]:
dfnumsongstot = dfnumsongs.merge(dfnumsongsskr, left_on=["main_artist"], right_on=["main_artist"], how="inner")

In [ ]:
dfnumsongstot.head(3)

#### Total Streams per Artist

In [ ]:
dfnumstreamsza = pd.DataFrame(zaplusskr.groupby("main_artist")["streams_za"].sum())
dfnumstreamsskr = pd.DataFrame(zaplusskr.groupby("main_artist")["streams_skr"].sum())
dfnumstreamsza = dfnumstreamsza.reset_index()
dfnumstreamsskr = dfnumstreamsskr.reset_index()

In [ ]:
dfnumsongsstreamsza = dfnumsongstot.merge(dfnumstreamsza, left_on=["main_artist"], right_on=["main_artist"], how="inner")
dfnumsongsstreamsza = dfnumsongsstreamsza.reset_index()
dfnumsongsstreamstot = dfnumsongsstreamsza.merge(dfnumstreamsskr, left_on=["main_artist"], right_on=["main_artist"], how="inner")
dfnumsongsstreamstot = dfnumsongsstreamstot.reset_index()

In [ ]:
dfnumsongsstreamstot.drop(["level_0", "index"], axis=1, inplace=True)

In [ ]:
dfnumsongsstreamstot

#### Num Appearances per Artist per song

In [ ]:
dfdatesza = pd.DataFrame(zaplusskr.groupby(["main_artist", "track_name_za"])["End Date_za"].nunique())
dfdatesskr = pd.DataFrame(zaplusskr.groupby(["main_artist", "track_name_skr"])["End Date_skr"].nunique())
dfdatesza = dfdatesza.reset_index()
dfdatesskr = dfdatesskr.reset_index()

In [ ]:
dfappearancesskr = pd.DataFrame(dfdatesskr.groupby("main_artist")["End Date_skr"].sum())
dfappearancesskr = dfappearancesskr.reset_index()
dfappearancesskr.columns = ["main_artist", "num_appearances_skr"]

dfappearancesza = pd.DataFrame(dfdatesza.groupby("main_artist")["End Date_za"].sum())
dfappearancesza = dfappearancesza.reset_index()
dfappearancesza.columns = ["main_artist", "num_appearances_za"]

In [ ]:
dfappearancesboth = dfappearancesza.merge(dfappearancesskr, left_on=["main_artist"], right_on=["main_artist"], how="inner")

In [ ]:
dfnumsongsstreamstotappearances = dfnumsongsstreamstot.merge(dfappearancesboth, left_on=["main_artist"], right_on=["main_artist"], how="inner")

#### Highest Position on the chart

In [ ]:
dfminrankza = pd.DataFrame(zaplusskr.groupby(["main_artist", "track_name_za"])["rank_za"].min())
dfminrankskr = pd.DataFrame(zaplusskr.groupby(["main_artist", "track_name_skr"])["rank_skr"].min())
dfminrankza = dfminrankza.reset_index()
dfminrankskr = dfminrankskr.reset_index()

In [ ]:
dfminrankartistza = pd.DataFrame(dfminrankza.groupby(["main_artist"])["rank_za"].min())
dfminrankartistza = dfminrankartistza.reset_index()

dfminrankartistskr = pd.DataFrame(dfminrankskr.groupby(["main_artist"])["rank_skr"].min())
dfminrankartistskr = dfminrankartistskr.reset_index()

In [ ]:
dfranksminartist = dfminrankartistza.merge(dfminrankartistskr, left_on=["main_artist"], right_on=["main_artist"], how="inner")

In [ ]:
dfnumsongsstreamstotappearancesranks = dfnumsongsstreamstotappearances.merge(dfranksminartist, left_on=["main_artist"], right_on=["main_artist"], how="inner")

In [ ]:
dfnumsongsstreamstotappearancesranks

#### Highest Position on the chart the songs

In [ ]:
dfminrankzavf = pd.DataFrame(dfnumsongsstreamstotappearancesranks.groupby("main_artist")["rank_za"].min())
dfminrankzavf = dfminrankzavf.reset_index()

dfmintracksza = dfminrankza.merge(dfminrankzavf, left_on=["main_artist", "rank_za"], right_on=["main_artist", "rank_za"], how="inner")
dfnummintracksza = pd.DataFrame(dfmintracksza.groupby("main_artist")["track_name_za"].nunique())
dfnummintracksza = dfnummintracksza.reset_index()
dfnummintracksza.columns = ["main_artist", "num_tracks_at_min_pos_za"]

In [ ]:
dfminrankskrvf = pd.DataFrame(dfnumsongsstreamstotappearancesranks.groupby("main_artist")["rank_skr"].min())
dfminrankskrvf = dfminrankskrvf.reset_index()

dfmintracksskr = dfminrankskr.merge(dfminrankskrvf, left_on=["main_artist", "rank_skr"], right_on=["main_artist", "rank_skr"], how="inner")
dfnummintracksskr = pd.DataFrame(dfmintracksskr.groupby("main_artist")["track_name_skr"].nunique())
dfnummintracksskr = dfnummintracksskr.reset_index()
dfnummintracksskr.columns = ["main_artist", "num_tracks_at_min_pos_skr"]

In [ ]:
dfnummintracksboth = dfnummintracksskr.merge(dfnummintracksza, left_on=["main_artist"], right_on=["main_artist"], how="inner")

In [ ]:
dfnummintracksboth

In [ ]:
dfnumsongsstreamstotappearancesranksname = dfnumsongsstreamstotappearancesranks.merge(dfnummintracksboth, left_on=["main_artist"], right_on=["main_artist"], how="inner")

In [ ]:
dfnumsongsstreamstotappearancesranksname.to_csv("Classification/dfnumsongsstreamstotappearancesranksname.csv", index="False")

In [ ]:
print("done.")